In [122]:
from __future__ import division
import os
import sys
import ipynb
import numpy as np
import pretty_midi
import argparse
import librosa

In [138]:
f = open("generatedSong.txt", "rb")
generatedSong = np.fromfile(f)
f.close
print("Before reshaping:", generatedSong.shape)
generatedSong = np.reshape(generatedSong, (128, 1000))
print("After reshaping:", generatedSong.shape)
print("Sample of data:", generatedSong[0])

Before reshaping: (128000,)
After reshaping: (128, 1000)
Sample of data: [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 3.15904617e-06
 6.85453415e-07 1.49011612e-07 3.87430191e-07 1.86562538e-05
 6.31809235e-06 8.31484795e-06 2.98023224e-08 6.49631023e-04
 0.00000000e+00 1.89125538e-04 1.66893005e-05 2.50339508e-06
 9.10582662e-01 1.81794167e-06 6.25848770e-06 4.79817390e-06
 2.31504440e-04 1.12026930e-04 2.75969505e-05 7.52707362e-01
 2.64048576e-05 3.91900539e-05 2.32934952e-04 9.68575478e-06
 1.13014877e-02 8.73804092e-05 9.15229321e-05 2.82824039e-05
 4.80747819e-02 1.85400248e-04 5.60820103e-04 9.34225678e-01
 2.43186951e

In [139]:
#for now I'm just accepting all values above 0.96
trimmedArray = np.where(generatedSong>0.90, generatedSong, 0)

In [140]:
#for now I'm just giving all nodes a velocity of 80
test = np.where(trimmedArray==0, trimmedArray, 80)

In [141]:
#Might need this later for proper assignements of velocity
#normalized = np.zeros(shape=(1, 128))
#for i in range(len(trimmedArray)):
#    temp = trimmedArray[i]
#    temp = np.reshape(temp, (1, 128))
#    arr = []
#    arr = np.where(temp==0, temp, i)
#    normalized = np.concatenate((normalized, arr))

In [142]:
#print(normalized.shape)
#print("Sample of timestamped array:", normalized[30])

In [143]:
def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                velocity=prev_velocities[note],
                pitch=note,
                start=note_on_time[note],
                end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [144]:
#test = np.rot90(test) #CHECK!!!
pm = piano_roll_to_pretty_midi(test, 5, 0)

In [145]:
pm.write('test2.mid')